# CSYE 7374: Introduction to Agentic AI - Final Project
# Education Content System

1. Install Dependencies

In [1]:
# Installing dependencies from requiremnts.txt in GitHub
!pip install -r https://raw.githubusercontent.com/ishreyasp/education_content_system/refs/heads/main/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.8/698.8 kB 17.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.4/237.4 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 27.8 MB/s eta 0:00:00
   ━━

2. Education Content Agents Pipeline

In [ ]:
%%writefile education_system.py

#Imports
import streamlit as st
import os
import json
import time
import tempfile
from datetime import datetime
from typing import List, Dict, Optional, Literal
from enum import Enum
import openai
from pydantic import BaseModel, Field
import faiss
import numpy as np
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
import PyPDF2
import docx
import re
import random
import hashlib
import sqlite3
from pathlib import Path

# ======================== CONFIGURATION ========================
class Config:
    """System configuration"""
    MAX_QUIZ_QUESTIONS = 10
    MIN_QUIZ_QUESTIONS = 3
    DEFAULT_CHUNK_SIZE = 1000
    DEFAULT_CHUNK_OVERLAP = 200
    COST_PER_CALL_GPT35 = 0.002
    COST_PER_CALL_GPT4 = 0.01
    DATABASE_PATH = "quiz_system.db"

# ======================== USER ROLES ========================
class UserRole(str, Enum):
    PROFESSOR = "professor"
    STUDENT = "student"

# ======================== AUTHENTICATION SYSTEM ========================
class AuthenticationManager:
    """Handle user authentication and session management"""
    
    def __init__(self):
        self.db_path = Config.DATABASE_PATH
        self.init_database()
        self.create_default_users()
    
    def init_database(self):
        """Initialize the database with user tables"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # Create users table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS users (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                username TEXT UNIQUE NOT NULL,
                password_hash TEXT NOT NULL,
                role TEXT NOT NULL,
                full_name TEXT NOT NULL,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        
        # Create quiz sessions table for cross-browser support
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS quiz_sessions (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                session_id TEXT UNIQUE NOT NULL,
                quiz_data TEXT NOT NULL,
                created_by TEXT NOT NULL,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                expires_at TIMESTAMP NOT NULL
            )
        ''')
        
        conn.commit()
        conn.close()
    
    def create_default_users(self):
        """Create default users for demonstration"""
        default_users = [
            ("prof_smith", "password123", "professor", "Professor Smith"),
            ("prof_jones", "password123", "professor", "Professor Jones"),
            ("student_alice", "password123", "student", "Alice Johnson"),
            ("student_bob", "password123", "student", "Bob Wilson"),
            ("student_charlie", "password123", "student", "Charlie Brown")
        ]
        
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        for username, password, role, full_name in default_users:
            try:
                password_hash = self.hash_password(password)
                cursor.execute('''
                    INSERT OR IGNORE INTO users (username, password_hash, role, full_name)
                    VALUES (?, ?, ?, ?)
                ''', (username, password_hash, role, full_name))
            except:
                pass  # User already exists
        
        conn.commit()
        conn.close()
    
    def hash_password(self, password: str) -> str:
        """Hash password using SHA-256"""
        return hashlib.sha256(password.encode()).hexdigest()
    
    def authenticate_user(self, username: str, password: str) -> Optional[Dict]:
        """Authenticate user and return user info if successful"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        password_hash = self.hash_password(password)
        cursor.execute('''
            SELECT username, role, full_name FROM users 
            WHERE username = ? AND password_hash = ?
        ''', (username, password_hash))
        
        result = cursor.fetchone()
        conn.close()
        
        if result:
            return {
                "username": result[0],
                "role": result[1],
                "full_name": result[2]
            }
        return None
    
    def get_all_users(self) -> List[Dict]:
        """Get all users for admin purposes"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute('SELECT username, role, full_name, created_at FROM users ORDER BY role, username')
        results = cursor.fetchall()
        conn.close()
        
        return [{"username": r[0], "role": r[1], "full_name": r[2], "created_at": r[3]} for r in results]
    
    def store_quiz_session(self, quiz_data: Dict, created_by: str) -> str:
        """Store quiz data for cross-browser access"""
        import uuid
        from datetime import datetime, timedelta
        
        session_id = str(uuid.uuid4())
        expires_at = datetime.now() + timedelta(hours=24)  
        
        # Convert QuizContent object to JSON-serializable format
        if 'quiz' in quiz_data and hasattr(quiz_data['quiz'], 'model_dump'):
            # Convert Pydantic model to dictionary
            serializable_quiz_data = quiz_data.copy()
            serializable_quiz_data['quiz'] = quiz_data['quiz'].model_dump()
            
            # Convert datetime objects to ISO format strings
            if 'created_at' in serializable_quiz_data['quiz']:
                created_at = serializable_quiz_data['quiz']['created_at']
                if hasattr(created_at, 'isoformat'):
                    serializable_quiz_data['quiz']['created_at'] = created_at.isoformat()
        else:
            serializable_quiz_data = quiz_data
        
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute('''
            INSERT INTO quiz_sessions (session_id, quiz_data, created_by, expires_at)
            VALUES (?, ?, ?, ?)
        ''', (session_id, json.dumps(serializable_quiz_data), created_by, expires_at))
        
        conn.commit()
        conn.close()
        
        return session_id
    
    def get_quiz_session(self, session_id: str) -> Optional[Dict]:
        """Retrieve quiz data by session ID"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute('''
            SELECT quiz_data, created_by FROM quiz_sessions 
            WHERE session_id = ? AND expires_at > datetime('now')
        ''', (session_id,))
        
        result = cursor.fetchone()
        conn.close()
        
        if result:
            quiz_data = json.loads(result[0])
            
            # Convert datetime string back to datetime object if needed
            if 'quiz' in quiz_data and 'created_at' in quiz_data['quiz']:
                created_at_str = quiz_data['quiz']['created_at']
                if isinstance(created_at_str, str):
                    try:
                        from datetime import datetime
                        quiz_data['quiz']['created_at'] = datetime.fromisoformat(created_at_str.replace('Z', '+00:00'))
                    except:
                        quiz_data['quiz']['created_at'] = datetime.now()
            
            return {
                "quiz_data": quiz_data,
                "created_by": result[1]
            }
        return None

# ======================== LLM MANAGEMENT ========================
class LLMInterface:
    """Centralized LLM management - tracks every call for cost analysis"""

    def __init__(self, api_key: str, model: str = "gpt-3.5-turbo"):
        self.client = openai.OpenAI(api_key=api_key.strip())
        self.model = model
        self.call_count = 0
        self.total_cost = 0.0

    def make_call(self, messages: List[Dict], response_format=None, temperature=0.3, max_retries=3):
        """Single point for ALL LLM calls with proper error handling and retries"""
        for attempt in range(max_retries):
            try:
                self.call_count += 1
                cost = Config.COST_PER_CALL_GPT35 if "gpt-3.5" in self.model else Config.COST_PER_CALL_GPT4
                self.total_cost += cost

                request_params = {
                    "model": self.model,
                    "messages": messages,
                    "temperature": temperature,
                    "max_tokens": 2000
                }

                if response_format and isinstance(response_format, dict):
                    request_params["response_format"] = response_format

                response = self.client.chat.completions.create(**request_params)
                return {"success": True, "content": response.choices[0].message.content}

            except Exception as e:
                if attempt == max_retries - 1:  # Last attempt
                    return {"success": False, "error": str(e)}
                time.sleep(2 ** attempt)  # Exponential backoff
        
        return {"success": False, "error": "Max retries exceeded"}

    def get_metrics(self) -> Dict:
        """Return usage metrics"""
        return {
            "total_calls": self.call_count,
            "total_cost": round(self.total_cost, 4),
            "average_cost_per_call": round(self.total_cost / max(1, self.call_count), 4)
        }

# ======================== DATA MODELS ========================
class QuestionDifficulty(str, Enum):
    EASY = "easy"
    MEDIUM = "medium"
    HARD = "hard"

class QuizQuestion(BaseModel):
    question: str = Field(description="The quiz question")
    options: List[str] = Field(description="Four multiple choice options", min_length=4, max_length=4)
    correct_answer: int = Field(description="Index of correct answer (0-3)", ge=0, le=3)
    explanation: str = Field(description="Brief explanation of the correct answer")
    difficulty: QuestionDifficulty = Field(default=QuestionDifficulty.MEDIUM)

class QuizContent(BaseModel):
    questions: List[QuizQuestion] = Field(description="List of quiz questions")
    topic: str = Field(description="Main topic of the quiz")
    total_questions: int = Field(description="Number of questions")
    difficulty_level: QuestionDifficulty = Field(description="Overall quiz difficulty set by professor")
    created_by: str = Field(description="Professor who created the quiz")
    created_at: datetime = Field(description="When the quiz was created")

# ======================== AGENTS (Same as before) ========================

class DocumentProcessorAgent:
    """ Agent 1: Document Processor: Accepts document with .pdf, .txt and .docx extention. Creates chunks of document content using langchain. """
    def __init__(self):
        self.name = "DocumentProcessor"
        self.uses_llm = False

    def extract_text_from_file(self, file_path: str, file_type: str) -> Dict:
        try:
            if file_type == "pdf":
                text = self._extract_from_pdf(file_path)
            elif file_type == "docx":
                text = self._extract_from_docx(file_path)
            elif file_type == "txt":
                text = self._extract_from_txt(file_path)
            else:
                return {"success": False, "error": f"Unsupported file type: {file_type}"}
            
            if not text.strip():
                return {"success": False, "error": "No content extracted from document"}
            
            return {"success": True, "text": text}
            
        except Exception as e:
            return {"success": False, "error": f"Document processing failed: {str(e)}"}

    def validate_content_quality(self, text: str) -> Dict:
        """Validate if document has enough content for educational quiz generation"""
        if not text or not text.strip():
            return {
                "is_valid": False,
                "error": "No content found in document",
                "suggestion": "Please upload a document with actual text content."
            }

        # Count sentences using multiple sentence endings
        import re
        cleaned_text = text.strip()
        sentence_pattern = r'[.!?]+[\s\n]+|[.!?]+$'
        sentences = re.split(sentence_pattern, cleaned_text)
        meaningful_sentences = [s.strip() for s in sentences if s.strip() and len(s.strip()) > 10]
        sentence_count = len(meaningful_sentences)

        # Count words for additional validation
        words = cleaned_text.split()
        word_count = len(words)

        # Stricter validation criteria
        if sentence_count < 5:
            return {
                "is_valid": False,
                "error": f"Document contains {sentence_count} meaningful sentence(s)",
                "suggestion": "Educational content should have at least 5 sentences to generate a proper quiz. Please upload a more comprehensive document."
            }

        if word_count < 100:
            return {
                "is_valid": False,
                "error": f"Document contains only {word_count} words",
                "suggestion": "Educational content should have at least 100 words to generate meaningful quiz questions. Please upload a longer document."
            }

        return {
            "is_valid": True,
            "sentence_count": sentence_count,
            "word_count": word_count,
            "message": f"Document validated: {sentence_count} sentences, {word_count} words"
        }

    def _extract_from_pdf(self, file_path: str) -> str:
        text = ""
        try:
            with open(file_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                for page in pdf_reader.pages:
                    text += page.extract_text() + "\n"
        except Exception as e:
            raise Exception(f"PDF extraction failed: {e}")
        return text

    def _extract_from_docx(self, file_path: str) -> str:
        try:
            doc = docx.Document(file_path)
            text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
            return text
        except Exception as e:
            raise Exception(f"DOCX extraction failed: {e}")

    def _extract_from_txt(self, file_path: str) -> str:
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                return file.read()
        except Exception as e:
            raise Exception(f"TXT extraction failed: {e}")

    def chunk_text(self, text: str) -> List[str]:
        if not text.strip():
            return []

        splitter = RecursiveCharacterTextSplitter(
            chunk_size=Config.DEFAULT_CHUNK_SIZE,
            chunk_overlap=Config.DEFAULT_CHUNK_OVERLAP,
            length_function=len
        )
        return splitter.split_text(text)

class VectorStore:
    """ Utility to create emdeddings for chunks and to store them in FAISS """
    def __init__(self, api_key: str):
        self.name = "VectorStore"
        self.uses_llm = False

        try:
            self.embeddings = OpenAIEmbeddings(
                openai_api_key=api_key,
                model="text-embedding-ada-002"
            )
            self.vector_store = None
        except Exception as e:
            self.embeddings = None

    def create_vector_store(self, text_chunks: List[str]) -> Dict:
        if not text_chunks:
            return {"success": False, "error": "No text chunks provided"}
        
        if not self.embeddings:
            return {"success": False, "error": "Embeddings not initialized"}

        try:
            self.vector_store = FAISS.from_texts(
                texts=text_chunks,
                embedding=self.embeddings
            )
            return {"success": True, "message": f"Vector store created with {len(text_chunks)} chunks"}
        except Exception as e:
            return {"success": False, "error": f"Vector store creation failed: {str(e)}"}

    def retrieve_relevant_content(self, query: str, k: int = 5) -> List[str]:
        if not self.vector_store:
            return []

        try:
            docs = self.vector_store.similarity_search(query, k=k)
            return [doc.page_content for doc in docs]
        except Exception as e:
            return []

class ContentAnalyzerAgent:
    """ Agent 2: Content Analyzer: Sends chunks of content to LLM to get the details of the content like topic, key concepts and difficulty level. """

    def __init__(self, llm_interface: LLMInterface):
        self.name = "ContentAnalyzer"
        self.uses_llm = True
        self.llm = llm_interface

    def analyze_content_for_quiz(self, content_chunks: List[str]) -> Dict:
        if not content_chunks:
            return {"success": False, "error": "No content chunks provided for analysis"}

        combined_content = "\n".join(content_chunks[:3])

        messages = [{
            "role": "system",
            "content": """Analyze the educational content and identify:
            1. Main topic/subject
            2. Key concepts that could be tested
            3. Content complexity level

            Respond in JSON format with keys: topic, key_concepts (array), complexity (easy/medium/hard)."""
        }, {
            "role": "user",
            "content": f"Analyze this educational content:\n\n{combined_content[:2000]}"
        }]

        response = self.llm.make_call(messages, {"type": "json_object"})
        
        if not response["success"]:
            return {"success": False, "error": f"LLM analysis failed: {response['error']}"}

        try:
            analysis = json.loads(response["content"])
            return {
                "success": True,
                "topic": analysis.get("topic", "Educational Content"),
                "key_concepts": analysis.get("key_concepts", []),
                "complexity": analysis.get("complexity", "medium")
            }
        except json.JSONDecodeError as e:
            return {"success": False, "error": f"Failed to parse analysis JSON: {str(e)}"}

class QuizGeneratorAgent:
    """ Agent 3: Quiz Generator: Creates multiple choice questions for given content. """

    def __init__(self, llm_interface: LLMInterface):
        self.name = "QuizGenerator"
        self.uses_llm = True
        self.llm = llm_interface

    def generate_quiz(self, content_chunks: List[str], analysis: Dict, num_questions: int = 5, 
                     difficulty_level: str = "medium", professor_name: str = "Professor") -> Dict:
        if not content_chunks:
            return {"success": False, "error": "No content chunks provided for quiz generation"}

        if not analysis.get("success", False):
            return {"success": False, "error": "Content analysis failed - cannot generate quiz"}

        relevant_content = "\n\n".join(content_chunks[:5])
        topic = analysis.get("topic", "Educational Content")

        # Enhanced prompt with better error handling
        messages = [{
            "role": "system",
            "content": f"""You are an expert educator creating a multiple-choice quiz.

            Create EXACTLY {num_questions} multiple-choice questions based on the provided content.
            
            IMPORTANT: All questions should be at {difficulty_level.upper()} difficulty level.
            
            Difficulty Guidelines:
            - EASY: Basic recall, definitions, simple concepts
            - MEDIUM: Understanding, application, connections between concepts  
            - HARD: Analysis, synthesis, complex reasoning, edge cases
            
            Each question MUST have:
            - A clear, specific question appropriate for {difficulty_level} level
            - Exactly 4 options (A, B, C, D)
            - One correct answer (index 0-3)
            - A brief explanation of why the answer is correct
            - Difficulty level matching the requested {difficulty_level} level

            Focus on testing understanding of the provided content, not general knowledge.

            CRITICAL: You must generate exactly {num_questions} questions. Do not generate fewer.

            Respond in JSON format matching this EXACT structure:
            {{
                "questions": [
                    {{
                        "question": "What is...?",
                        "options": ["Option A", "Option B", "Option C", "Option D"],
                        "correct_answer": 0,
                        "explanation": "Brief explanation...",
                        "difficulty": "{difficulty_level}"
                    }}
                ],
                "topic": "{topic}",
                "total_questions": {num_questions}
            }}"""
        }, {
            "role": "user",
            "content": f"Create a {num_questions}-question {difficulty_level} difficulty quiz from this content:\n\n{relevant_content[:3000]}"
        }]

        response = self.llm.make_call(messages, {"type": "json_object"})
        
        if not response["success"]:
            return {"success": False, "error": f"Quiz generation LLM call failed: {response['error']}"}

        try:
            quiz_data = json.loads(response["content"])
            
            # Validate the response structure
            if "questions" not in quiz_data:
                return {"success": False, "error": "LLM response missing 'questions' field"}
            
            questions = quiz_data["questions"]
            if not questions or len(questions) == 0:
                return {"success": False, "error": "LLM generated no questions"}
            
            if len(questions) < num_questions:
                return {"success": False, "error": f"LLM generated only {len(questions)} questions, expected {num_questions}"}
            
            # Validate each question structure
            for i, q in enumerate(questions):
                required_fields = ["question", "options", "correct_answer", "explanation"]
                for field in required_fields:
                    if field not in q:
                        return {"success": False, "error": f"Question {i+1} missing field: {field}"}
                
                if not isinstance(q["options"], list) or len(q["options"]) != 4:
                    return {"success": False, "error": f"Question {i+1} must have exactly 4 options"}
                
                if not isinstance(q["correct_answer"], int) or q["correct_answer"] < 0 or q["correct_answer"] > 3:
                    return {"success": False, "error": f"Question {i+1} has invalid correct_answer index"}
            
            # Create quiz object with all required fields
            quiz_data_complete = {
                "questions": quiz_data["questions"],
                "topic": quiz_data.get("topic", "Educational Content"),
                "total_questions": quiz_data.get("total_questions", len(quiz_data["questions"])),
                "difficulty_level": difficulty_level,
                "created_by": professor_name,
                "created_at": datetime.now()
            }
            
            quiz = QuizContent(**quiz_data_complete)
            
            return {"success": True, "quiz": quiz}

        except json.JSONDecodeError as e:
            return {"success": False, "error": f"Failed to parse quiz JSON: {str(e)}"}
        except Exception as e:
            return {"success": False, "error": f"Quiz creation failed: {str(e)}"}

# ======================== HYBRID SYSTEM COORDINATOR ========================

class HybridEducationalSystem:
    """ Centralized Agent Coordinator """
    def __init__(self, api_key: str):
        self.llm = LLMInterface(api_key)
        self.document_processor = DocumentProcessorAgent()
        self.vector_store_agent = VectorStore(api_key)
        self.content_analyzer = ContentAnalyzerAgent(self.llm)
        self.quiz_generator = QuizGeneratorAgent(self.llm)

    def process_document(self, file_path: str, file_type: str, num_questions: int = 5, 
                        difficulty_level: str = "medium", professor_name: str = "Professor") -> Dict:
        start_time = time.time()

        # Create status containers for real-time updates
        status_container = st.empty()
        progress_bar = st.progress(0)

        try:
            # Step 1: Document Processing Agent
            status_container.info("🤖 **Agent 1: DocumentProcessorAgent** - Extracting text from your document...")
            progress_bar.progress(0.15)

            extraction_result = self.document_processor.extract_text_from_file(file_path, file_type)
            
            if not extraction_result["success"]:
                status_container.error(f"❌ **Document Extraction Failed** - {extraction_result['error']}")
                progress_bar.progress(0)
                return {
                    "success": False,
                    "error": "document_extraction_failed",
                    "details": extraction_result["error"],
                    "metrics": self.llm.get_metrics()
                }

            text_content = extraction_result["text"]

            # Step 1.5 - Validate content quality
            status_container.info("🔍 **Validating Content Quality** - Checking if document is suitable for quiz generation...")
            progress_bar.progress(0.25)

            validation_result = self.document_processor.validate_content_quality(text_content)

            if not validation_result["is_valid"]:
                status_container.error(f"❌ **Content Validation Failed** - {validation_result['error']}")
                progress_bar.progress(0)
                return {
                    "success": False,
                    "error": "insufficient_content",
                    "validation_error": validation_result["error"],
                    "suggestion": validation_result["suggestion"],
                    "metrics": self.llm.get_metrics()
                }

            status_container.success(f"✅ **Content Validated** - {validation_result['message']}")

            chunks = self.document_processor.chunk_text(text_content)
            if not chunks:
                status_container.error("❌ **Text Chunking Failed** - Unable to create content chunks")
                progress_bar.progress(0)
                return {
                    "success": False,
                    "error": "chunking_failed",
                    "details": "No text chunks could be created",
                    "metrics": self.llm.get_metrics()
                }

            status_container.success(f"✅ **Agent 1 Complete** - Created {len(chunks)} text chunks (0 LLM calls)")
            time.sleep(0.5)

            # Step 2: Vector Store Agent
            status_container.info("🔍 **VectorStore** - Creating semantic search index...")
            progress_bar.progress(0.40)

            vector_result = self.vector_store_agent.create_vector_store(chunks)
            if not vector_result["success"]:
                status_container.error(f"❌ **Vector Store Creation Failed** - {vector_result['error']}")
                progress_bar.progress(0)
                return {
                    "success": False,
                    "error": "vector_store_failed",
                    "details": vector_result["error"],
                    "metrics": self.llm.get_metrics()
                }

            status_container.success("✅ **Vector embeddings ready** (0 LLM calls)")
            time.sleep(0.5)

            # Step 3: Content Analyzer Agent
            status_container.info("🤖 **Agent 2: ContentAnalyzerAgent** - Analyzing content structure...")
            progress_bar.progress(0.65)

            relevant_chunks = self.vector_store_agent.retrieve_relevant_content("main concepts key topics", k=5)
            if not relevant_chunks:
                status_container.error("❌ **Content Retrieval Failed** - No relevant content chunks found")
                progress_bar.progress(0)
                return {
                    "success": False,
                    "error": "content_retrieval_failed",
                    "details": "Unable to retrieve relevant content for analysis",
                    "metrics": self.llm.get_metrics()
                }

            analysis_result = self.content_analyzer.analyze_content_for_quiz(relevant_chunks)
            
            if not analysis_result["success"]:
                status_container.error(f"❌ **Content Analysis Failed** - {analysis_result['error']}")
                progress_bar.progress(0)
                return {
                    "success": False,
                    "error": "content_analysis_failed",
                    "details": analysis_result["error"],
                    "metrics": self.llm.get_metrics()
                }

            status_container.success(f"✅ **Agent 2 Complete** - Identified topic: '{analysis_result['topic']}' (1 LLM call)")
            time.sleep(0.5)

            # Step 4: Quiz Generator Agent
            status_container.info(f"🤖 **Agent 3: QuizGeneratorAgent** - Creating {difficulty_level.upper()} difficulty questions...")
            progress_bar.progress(0.90)

            quiz_chunks = self.vector_store_agent.retrieve_relevant_content("important information facts", k=5)
            if not quiz_chunks:
                status_container.error("❌ **Quiz Content Retrieval Failed** - No content available for quiz generation")
                progress_bar.progress(0)
                return {
                    "success": False,
                    "error": "quiz_content_retrieval_failed",
                    "details": "Unable to retrieve content for quiz questions",
                    "metrics": self.llm.get_metrics()
                }

            quiz_result = self.quiz_generator.generate_quiz(quiz_chunks, analysis_result, num_questions, difficulty_level, professor_name)
            
            if not quiz_result["success"]:
                status_container.error(f"❌ **Quiz Generation Failed** - {quiz_result['error']}")
                progress_bar.progress(0)
                return {
                    "success": False,
                    "error": "quiz_generation_failed",
                    "details": quiz_result["error"],
                    "metrics": self.llm.get_metrics()
                }

            # Calculate metrics
            processing_time = time.time() - start_time
            metrics = self.llm.get_metrics()

            # Final status
            progress_bar.progress(1.0)
            status_container.success(f"🎉 **All Agents Complete!** - Created {difficulty_level.upper()} quiz with {metrics['total_calls']} LLM calls, ${metrics['total_cost']:.4f} cost, {processing_time:.1f}s")

            return {
                "success": True,
                "quiz": quiz_result["quiz"],
                "analysis": analysis_result,
                "processing_time": round(processing_time, 2),
                "metrics": metrics
            }

        except Exception as e:
            status_container.error(f"❌ **Processing Failed** - {str(e)}")
            progress_bar.progress(0)
            return {
                "success": False,
                "error": "unexpected_error",
                "details": str(e),
                "metrics": self.llm.get_metrics()
            }

# ======================== AUTHENTICATION UI ========================

def display_login_interface(auth_manager: AuthenticationManager):
    """Display login interface"""
    st.markdown("## 🔐 Login to Quiz System")
    
    # Check if we're accessing a shared quiz
    try:
        # Try the newer method first
        query_params = st.query_params
    except AttributeError:
        # Fall back to the older method
        query_params = st.experimental_get_query_params()
    
    if 'quiz_session' in query_params:
        session_id = query_params['quiz_session']
        if isinstance(session_id, list):
            session_id = session_id[0]  # Handle list format from older Streamlit versions
        session_data = auth_manager.get_quiz_session(session_id)
        
        if session_data:
            st.info(f"📝 **Accessing Shared Quiz** created by {session_data['created_by']}")
            st.markdown("Please login to take the quiz:")
        else:
            st.error("❌ **Invalid or Expired Quiz Link**")
            st.markdown("The quiz session has expired or the link is invalid.")
            return None, None
    
    # Create login form
    with st.container():
        col1, col2, col3 = st.columns([1, 2, 1])
        
        with col2:
            st.markdown("""
            <div style="background: white; padding: 2rem; border-radius: 10px; box-shadow: 0 4px 12px rgba(0,0,0,0.1);">
            """, unsafe_allow_html=True)
            
            with st.form("login_form"):
                st.markdown("### Please Login")
                
                username = st.text_input("👤 Username:", placeholder="Enter your username")
                password = st.text_input("🔒 Password:", type="password", placeholder="Enter your password")
                
                submit_button = st.form_submit_button("🚀 Login", type="primary", use_container_width=True)
                
                if submit_button:
                    if not username or not password:
                        st.error("❌ Please enter both username and password")
                    else:
                        user_info = auth_manager.authenticate_user(username, password)
                        if user_info:
                            # Store user info in session state
                            st.session_state.authenticated = True
                            st.session_state.user_info = user_info
                            st.success(f"✅ Welcome, {user_info['full_name']}!")
                            
                            # Check if accessing shared quiz
                            try:
                                # Try the newer method first
                                query_params = st.query_params
                            except AttributeError:
                                # Fall back to the older method
                                query_params = st.experimental_get_query_params()
                            
                            if 'quiz_session' in query_params:
                                session_id = query_params['quiz_session']
                                if isinstance(session_id, list):
                                    session_id = session_id[0]  # Handle list format
                                session_data = auth_manager.get_quiz_session(session_id)
                                if session_data:
                                    st.session_state.shared_quiz_data = session_data
                            
                            time.sleep(1)
                            st.rerun()
                        else:
                            st.error("❌ Invalid username or password")
            
            st.markdown("</div>", unsafe_allow_html=True)
    
    # Demo accounts information
    st.markdown("---")
    st.markdown("### 🎯 Demo Accounts")
    
    col1, col2 = st.columns(2)
    
    with col1:
        st.markdown("""
        **👨‍🏫 Professor Accounts:**
        - Username: `prof_smith` | Password: `password123`
        - Username: `prof_jones` | Password: `password123`
        """)
    
    with col2:
        st.markdown("""
        **👨‍🎓 Student Accounts:**
        - Username: `student_alice` | Password: `password123`
        - Username: `student_bob` | Password: `password123`
        - Username: `student_charlie` | Password: `password123`
        """)
    
    return None, None

def display_logout_button():
    """Display logout button in sidebar"""
    if st.sidebar.button("🚪 Logout", type="secondary"):
        # Clear all session state
        for key in list(st.session_state.keys()):
            del st.session_state[key]
        st.rerun()

def display_user_info():
    """Display current user info in sidebar"""
    if 'user_info' in st.session_state:
        user_info = st.session_state.user_info
        role_emoji = "👨‍🏫" if user_info['role'] == 'professor' else "👨‍🎓"
        
        st.sidebar.markdown("### 👤 Current User")
        st.sidebar.markdown(f"""
        {role_emoji} **{user_info['full_name']}**  
        📋 Role: {user_info['role'].title()}  
        🔑 Username: {user_info['username']}
        """)

# ======================== ROLE-BASED INTERFACES ========================

def display_professor_interface(edu_system, auth_manager):
    """Professor interface for creating quizzes"""
    st.markdown("## 👨‍🏫 Professor Dashboard")
    st.markdown("Upload educational content to generate interactive quizzes for your students.")
    
    # Quiz creation section
    with st.container():
        st.markdown("### 📁 Create New Quiz")
        
        col1, col2 = st.columns([2, 1])
        
        with col1:
            uploaded_file = st.file_uploader(
                "Upload educational document:",
                type=['pdf', 'docx', 'txt'],
                help="Upload PDF, DOCX, or TXT files containing educational content"
            )
        
        with col2:
            num_questions = st.slider(
                "Number of questions:",
                min_value=Config.MIN_QUIZ_QUESTIONS,
                max_value=Config.MAX_QUIZ_QUESTIONS,
                value=5
            )
            
            difficulty_level = st.selectbox(
                "Quiz difficulty:",
                options=["easy", "medium", "hard"],
                index=1,
                format_func=lambda x: f"📈 {x.title()}"
            )
        
        if uploaded_file:
            st.success(f"📄 File ready: **{uploaded_file.name}**")
            
            # Quiz generation button
            if st.button("🚀 Generate Quiz for Students", type="primary", use_container_width=True):
                
                professor_name = st.session_state.user_info['full_name']
                
                # Save file temporarily
                with tempfile.NamedTemporaryFile(delete=False, suffix=f".{uploaded_file.name.split('.')[-1]}") as tmp_file:
                    tmp_file.write(uploaded_file.getvalue())
                    file_path = tmp_file.name
                
                file_type = uploaded_file.name.split('.')[-1].lower()
                
                # Process with selected difficulty
                with st.spinner(f"🔄 Creating {difficulty_level.upper()} difficulty quiz..."):
                    result = edu_system.process_document(
                        file_path=file_path, 
                        file_type=file_type, 
                        num_questions=num_questions, 
                        difficulty_level=difficulty_level, 
                        professor_name=professor_name
                    )
                
                os.unlink(file_path)
                
                if result['success']:
                    st.success("✅ Quiz created successfully!")
                    
                    # Store quiz for students
                    if 'available_quizzes' not in st.session_state:
                        st.session_state.available_quizzes = []
                    
                    quiz_info = {
                        'quiz': result['quiz'],
                        'metrics': result['metrics'],
                        'processing_time': result['processing_time'],
                        'file_name': uploaded_file.name
                    }
                    st.session_state.available_quizzes.append(quiz_info)
                    
                    # Store quiz in database for cross-browser access
                    session_id = auth_manager.store_quiz_session(
                        quiz_info, 
                        st.session_state.user_info['username']
                    )
                    
                    # Show success message with quiz details and shareable link
                    st.balloons()
                    st.info(f"""
                    📊 **Quiz Details:**
                    - **Topic**: {result['quiz'].topic}
                    - **Questions**: {len(result['quiz'].questions)}
                    - **Difficulty**: {difficulty_level.title()}
                    - **Created by**: {professor_name}
                    - **Cost**: ${result['metrics']['total_cost']:.4f}
                    """)
                    
                    # Generate shareable link using actual ngrok URL if available
                    try:
                        # Try to get the current ngrok URL from session or environment
                        if 'ngrok_url' in st.session_state:
                            base_url = st.session_state.ngrok_url
                            shareable_link = f"{base_url}/?quiz_session={session_id}"
                        else:
                            # Check if we can detect ngrok
                            try:
                                import requests
                                # Try to get ngrok tunnels
                                response = requests.get("http://localhost:4040/api/tunnels")
                                tunnels = response.json()
                                if tunnels.get('tunnels'):
                                    public_url = tunnels['tunnels'][0]['public_url']
                                    base_url = public_url
                                    st.session_state.ngrok_url = base_url  # Store for future use
                                    shareable_link = f"{base_url}/?quiz_session={session_id}"
                                else:
                                    raise Exception("No tunnels found")
                            except:
                                base_url = "[YOUR_NGROK_URL]"
                                shareable_link = f"{base_url}/?quiz_session={session_id}"
                    except:
                        base_url = "[YOUR_NGROK_URL]"
                        shareable_link = f"{base_url}/?quiz_session={session_id}"
                    
                    st.markdown("### 🔗 Share Quiz with Students")
                    
                    if base_url == "[YOUR_NGROK_URL]":
                        st.warning("⚠️ Replace `[YOUR_NGROK_URL]` with your actual ngrok URL")
                        st.markdown(f"**Your ngrok URL format:** `https://xxxxx.ngrok-free.app`")
                    
                    st.code(shareable_link)
                    
                    # Show the session ID separately
                    st.markdown("### 🔑 Session ID")
                    st.code(session_id)
                    
                    # Instructions
                    st.markdown("""
                    **📋 Instructions for Students:**
                    1. **Direct Link**: Use the complete URL above
                    2. **Manual**: Go to your app URL and add `?quiz_session=SESSION_ID`
                    3. **Login**: Use student credentials (e.g., `student_alice` / `password123`)
                    
                    **⚠️ Note**: Students might see an ngrok warning page first - they should click "Visit Site"
                    """)
                    
                else:
                    # Handle errors with specific messages and helpful suggestions
                    st.error("❌ **Quiz Generation Failed**")
                    
                    error_type = result.get('error', 'unknown_error')
                    error_details = result.get('details', 'Unknown error occurred')
                    
                    if error_type == 'insufficient_content':
                        st.warning(f"**Issue**: {result['validation_error']}")
                        st.info(f"**Suggestion**: {result['suggestion']}")
                    elif error_type == 'document_extraction_failed':
                        st.error(f"**Document Processing Error**: {error_details}")
                        st.info("**Suggestions**: Try a different file format or check if the file is corrupted")
                    elif error_type == 'content_analysis_failed':
                        st.error(f"**Content Analysis Error**: {error_details}")
                        st.info("**Suggestions**: Check your internet connection or try with different content")
                    elif error_type == 'quiz_generation_failed':
                        st.error(f"**Quiz Generation Error**: {error_details}")
                        st.info("**Suggestions**: Try reducing the number of questions or using different content")
                    else:
                        st.error(f"**Error Details**: {error_details}")
                    
                    st.markdown("""
                    **General troubleshooting:**
                    - Ensure your document has sufficient educational content
                    - Check your internet connection for API calls
                    - Try reducing the number of questions requested
                    - Verify the document file is not corrupted
                    - Upload content with clear educational structure
                    """)
                        
                    # Show metrics even for failed attempts
                    if 'metrics' in result:
                        st.markdown("### 📊 Processing Metrics")
                        display_metrics(result['metrics'], 0)

def display_student_interface(auth_manager):
    """Student interface for taking quizzes"""
    st.markdown("## 👨‍🎓 Student Dashboard")
    
    # Check if accessing a shared quiz
    if 'shared_quiz_data' in st.session_state:
        st.info(f"📝 **Shared Quiz** from {st.session_state.shared_quiz_data['created_by']}")
        quiz_data = st.session_state.shared_quiz_data['quiz_data']
        
        # Convert quiz data back to QuizContent object
        try:
            quiz_dict = quiz_data['quiz']
            
            # Ensure datetime is properly handled
            if 'created_at' in quiz_dict and isinstance(quiz_dict['created_at'], str):
                from datetime import datetime
                try:
                    quiz_dict['created_at'] = datetime.fromisoformat(quiz_dict['created_at'].replace('Z', '+00:00'))
                except:
                    quiz_dict['created_at'] = datetime.now()
            
            quiz = QuizContent(**quiz_dict)
            st.session_state.current_quiz = quiz
            st.session_state.quiz_mode = "taking"
            
            display_quiz(quiz)
            return
        except Exception as e:
            st.error(f"❌ **Failed to load shared quiz**: {str(e)}")
            st.markdown("Please try accessing the quiz link again or contact your professor.")
            return
    
    # Check if there are any available quizzes
    if 'available_quizzes' not in st.session_state or not st.session_state.available_quizzes:
        st.info("📚 **No quizzes available yet**")
        st.markdown("""
        Waiting for your professor to upload educational content and generate quizzes.
        
        **What you can expect:**
        - 🎯 Interactive multiple-choice questions
        - ⚡ Instant feedback on your answers
        - 💡 Detailed explanations for learning
        - 📊 Progress tracking and final scores
        """)
        return
    
    st.markdown("### 📋 Available Quizzes")
    
    # Display available quizzes
    for idx, quiz_info in enumerate(st.session_state.available_quizzes):
        quiz = quiz_info['quiz']
        
        with st.expander(f"📝 **{quiz.topic}** ({quiz.difficulty_level.value.title()} Level)", expanded=False):
            col1, col2 = st.columns([2, 1])
            
            with col1:
                st.markdown(f"""
                **📚 Quiz Information:**
                - **Created by**: {quiz.created_by}
                - **Questions**: {quiz.total_questions}
                - **Difficulty**: {quiz.difficulty_level.value.title()}
                - **Source**: {quiz_info['file_name']}
                """)
            
            with col2:
                if st.button(f"📝 Take Quiz", key=f"take_quiz_{idx}", type="primary"):
                    st.session_state.current_quiz = quiz
                    st.session_state.current_quiz_index = idx
                    st.session_state.quiz_mode = "taking"
                    st.rerun()
    
    # Display quiz if student is taking one
    if st.session_state.get('quiz_mode') == 'taking' and 'current_quiz' in st.session_state:
        st.markdown("---")
        st.markdown("### 🎯 Quiz in Progress")
        
        # Back to quiz list button
        if st.button("⬅️ Back to Quiz List", type="secondary"):
            st.session_state.quiz_mode = "browsing"
            st.session_state.pop('current_quiz', None)
            # Clear quiz state when going back
            for key in list(st.session_state.keys()):
                if key.startswith(('quiz_selections', 'quiz_submitted', 'quiz_show_results')):
                    del st.session_state[key]
            st.rerun()
        
        display_quiz(st.session_state.current_quiz)

# ======================== STREAMLIT UI ========================

def setup_colab_tunnel():
    """Setup instructions and helper for Colab tunneling"""
    if 'colab_tunnel_setup' not in st.session_state:
        st.session_state.colab_tunnel_setup = False

    """ Get API key """
    try:
        from google.colab import userdata
        api_key = userdata.get('OPENAI_API_KEY')
        return api_key
    except Exception as e:
        return None

def create_beautiful_ui():
    """ Setup beautiful UI """
    st.set_page_config(
        page_title="🎓 AI Quiz Generator",
        page_icon="🎓",
        layout="wide"
    )

    st.markdown("""
    <style>
    .main-header {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 50%, #f093fb 100%);
        padding: 2.5rem 2rem;
        border-radius: 20px;
        color: white;
        text-align: center;
        margin-bottom: 1.5rem;
        position: relative;
        overflow: hidden;
        box-shadow: 0 10px 40px rgba(102, 126, 234, 0.3);
    }

    .creative-title {
        font-size: 3.5rem;
        font-weight: 800;
        margin: 0;
        color: #ffffff;
        text-shadow: 0 4px 20px rgba(0,0,0,0.3);
        letter-spacing: -2px;
    }

    .metric-card {
        background: linear-gradient(145deg, #2d3748, #4a5568);
        border-radius: 16px;
        padding: 1.5rem;
        margin: 0.5rem;
        box-shadow: 0 8px 32px rgba(0, 0, 0, 0.3);
        border: 1px solid rgba(255, 255, 255, 0.1);
        position: relative;
        overflow: hidden;
    }

    .metric-card::before {
        content: '';
        position: absolute;
        top: 0;
        left: 0;
        right: 0;
        height: 3px;
        background: linear-gradient(90deg, #667eea, #764ba2);
    }

    .metric-icon {
        font-size: 1.8rem;
        margin-bottom: 0.5rem;
        display: block;
    }

    .metric-label {
        color: #a0aec0;
        font-size: 0.85rem;
        font-weight: 500;
        margin: 0;
        text-transform: uppercase;
        letter-spacing: 0.5px;
    }

    .metric-value {
        color: #ffffff;
        font-size: 1.8rem;
        font-weight: 700;
        margin: 0.3rem 0 0 0;
        line-height: 1.2;
    }

    .metric-trend {
        color: #68d391;
        font-size: 0.75rem;
        margin-top: 0.25rem;
    }
    
    .role-badge {
        display: inline-block;
        padding: 0.25rem 0.75rem;
        border-radius: 20px;
        font-size: 0.8rem;
        font-weight: 600;
        margin-left: 0.5rem;
    }
    
    .professor-badge {
        background: #667eea;
        color: white;
    }
    
    .student-badge {
        background: #48bb78;
        color: white;
    }
    </style>
    """, unsafe_allow_html=True)

def display_metrics(metrics: Dict, processing_time: float = 0):
    """Display beautiful performance metrics cards"""
    col1, col2, col3, col4 = st.columns(4)

    with col1:
        st.markdown(f"""
        <div class="metric-card">
            <div class="metric-icon">🔥</div>
            <p class="metric-label">Total LLM Calls</p>
            <h2 class="metric-value">{metrics.get('total_calls', 0)}</h2>
        </div>
        """, unsafe_allow_html=True)

    with col2:
        cost = metrics.get('total_cost', 0)
        st.markdown(f"""
        <div class="metric-card">
            <div class="metric-icon">💰</div>
            <p class="metric-label">Total Cost</p>
            <h2 class="metric-value">${cost:.4f}</h2>
        </div>
        """, unsafe_allow_html=True)

    with col3:
        avg_cost = metrics.get('average_cost_per_call', 0)
        st.markdown(f"""
        <div class="metric-card">
            <div class="metric-icon">📊</div>
            <p class="metric-label">Average Cost</p>
            <h2 class="metric-value">${avg_cost:.4f}</h2>
        </div>
        """, unsafe_allow_html=True)

    with col4:
        st.markdown(f"""
        <div class="metric-card">
            <div class="metric-icon">⏱️</div>
            <p class="metric-label">Processing Time</p>
            <h2 class="metric-value">{processing_time:.1f}s</h2>
        </div>
        """, unsafe_allow_html=True)

def display_quiz(quiz: QuizContent):
    """Display the interactive quiz with difficulty indication"""
    if not quiz or not quiz.questions:
        st.error("❌ No quiz questions generated")
        return

    # Quiz header with difficulty and creator info
    difficulty_color = {
        "easy": "#48bb78",
        "medium": "#ed8936", 
        "hard": "#e53e3e"
    }
    
    st.markdown(f"""
    <div style="background: white; padding: 2rem; border-radius: 12px; box-shadow: 0 4px 12px rgba(0,0,0,0.1); margin: 1rem 0;">
        <h2 style="color: #2d3748; margin: 0 0 1rem 0;">📝 Quiz: {quiz.topic}</h2>
        <div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 1rem;">
            <p style="color: #4a5568; margin: 0;"><strong>Total Questions:</strong> {len(quiz.questions)}</p>
            <div style="display: flex; gap: 1rem;">
                <span style="background: {difficulty_color.get(quiz.difficulty_level.value, '#ed8936')}; color: white; padding: 0.25rem 0.75rem; border-radius: 15px; font-size: 0.9rem; font-weight: 600;">
                    📈 {quiz.difficulty_level.value.title()} Level
                </span>
                <span style="background: #667eea; color: white; padding: 0.25rem 0.75rem; border-radius: 15px; font-size: 0.9rem;">
                    👨‍🏫 {quiz.created_by}
                </span>
            </div>
        </div>
    </div>
    """, unsafe_allow_html=True)

    # Initialize session state
    if 'quiz_selections' not in st.session_state:
        st.session_state.quiz_selections = {}
    if 'quiz_submitted' not in st.session_state:
        st.session_state.quiz_submitted = set()
    if 'quiz_show_results' not in st.session_state:
        st.session_state.quiz_show_results = {}

    # Display each question
    for i, question in enumerate(quiz.questions):
        st.markdown("---")
        st.markdown(f"### Question {i+1}")
        
        # Show question difficulty
        q_difficulty_color = difficulty_color.get(question.difficulty.value, '#ed8936')
        st.markdown(f"""
        <div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 1rem;">
            <span style="font-weight: 600; font-size: 1.1rem;">{question.question}</span>
            <span style="background: {q_difficulty_color}; color: white; padding: 0.2rem 0.6rem; border-radius: 12px; font-size: 0.8rem;">
                {question.difficulty.value.title()}
            </span>
        </div>
        """, unsafe_allow_html=True)

        question_submitted = i in st.session_state.quiz_submitted

        if not question_submitted:
            # Selection phase
            st.markdown("**Click on your answer:**")

            for j, option in enumerate(question.options):
                option_letter = chr(65 + j)
                option_text = f"{option_letter}. {option}"
                is_selected = st.session_state.quiz_selections.get(i) == j
                button_type = "primary" if is_selected else "secondary"
                button_label = f"{'🔘' if is_selected else '⚪'} {option_text}"

                if st.button(button_label, key=f"option_q{i}_choice{j}", type=button_type, use_container_width=True):
                    st.session_state.quiz_selections[i] = j
                    st.rerun()

            # Submit button
            if i in st.session_state.quiz_selections:
                selected_option = st.session_state.quiz_selections[i]
                selected_text = f"{chr(65+selected_option)}. {question.options[selected_option]}"
                st.info(f"💭 **You selected:** {selected_text}")

                col1, col2, col3 = st.columns([1, 1, 1])
                with col2:
                    if st.button("✅ Submit Answer", key=f"submit_btn_{i}", type="primary", use_container_width=True):
                        st.session_state.quiz_submitted.add(i)
                        st.session_state.quiz_show_results[i] = selected_option
                        st.rerun()
            else:
                st.warning("👆 Please select an option above")

        else:
            # Results phase
            user_answer = st.session_state.quiz_show_results[i]
            selected_text = f"{chr(65+user_answer)}. {question.options[user_answer]}"
            st.markdown(f"**Your answer:** {selected_text}")

            if user_answer == question.correct_answer:
                st.success("🎉 **Correct!**")
                st.info(f"💡 **Explanation:** {question.explanation}")
            else:
                correct_option = f"{chr(65+question.correct_answer)}. {question.options[question.correct_answer]}"
                st.error("❌ **Incorrect**")
                st.warning(f"✅ **Correct Answer:** {correct_option}")
                st.info(f"💡 **Explanation:** {question.explanation}")

            col1, col2, col3 = st.columns([1, 1, 1])
            with col2:
                if st.button("🔄 Try Again", key=f"retry_btn_{i}", type="secondary"):
                    st.session_state.quiz_submitted.discard(i)
                    if i in st.session_state.quiz_show_results:
                        del st.session_state.quiz_show_results[i]
                    st.rerun()

    # Progress and final score
    total_submitted = len(st.session_state.quiz_submitted)

    if total_submitted > 0:
        progress = total_submitted / len(quiz.questions)
        st.progress(progress)
        st.caption(f"📊 Progress: {total_submitted}/{len(quiz.questions)} questions completed")

        if total_submitted == len(quiz.questions):
            correct_count = sum(
                1 for i in range(len(quiz.questions))
                if st.session_state.quiz_show_results.get(i) == quiz.questions[i].correct_answer
            )

            score_percentage = (correct_count / len(quiz.questions)) * 100

            st.markdown("---")
            st.markdown("## 🏆 Quiz Complete!")

            col1, col2, col3 = st.columns([1, 2, 1])
            with col2:
                if score_percentage >= 80:
                    st.balloons()
                    st.success(f"🎉 **Outstanding!** {correct_count}/{len(quiz.questions)} ({score_percentage:.0f}%)")
                elif score_percentage >= 60:
                    st.info(f"👍 **Well Done!** {correct_count}/{len(quiz.questions)} ({score_percentage:.0f}%)")
                else:
                    st.warning(f"📚 **Keep Learning!** {correct_count}/{len(quiz.questions)} ({score_percentage:.0f}%)")

def main():
    """Main function with authentication and role-based access control"""

    create_beautiful_ui()
    
    # Initialize authentication manager
    if 'auth_manager' not in st.session_state:
        st.session_state.auth_manager = AuthenticationManager()
    
    auth_manager = st.session_state.auth_manager

    # Check if user is authenticated
    if not st.session_state.get('authenticated', False):
        # Show header for login page
        st.markdown(f"""
        <div class="main-header">
            <h1 class="creative-title">🎓 AI Quiz Generator</h1>
            <p style="font-size: 1.3rem; margin: 1rem 0 0 0;">Transform educational content into interactive learning experiences</p>
            <p style="font-size: 0.9rem; margin-top: 0.5rem; font-style: italic;">✨ Powered by Multi-Agent AI System • ⚡ Instant Feedback • 🎯 Personalized Quizzes</p>
        </div>
        """, unsafe_allow_html=True)
        
        display_login_interface(auth_manager)
        return

    # User is authenticated - show sidebar with user info and logout
    display_user_info()
    display_logout_button()
    
    # Show header with user role
    user_role = st.session_state.user_info['role']
    role_display = "👨‍🏫 Professor" if user_role == 'professor' else "👨‍🎓 Student"
    badge_class = "professor-badge" if user_role == 'professor' else "student-badge"

    st.markdown(f"""
    <div class="main-header">
        <h1 class="creative-title">🎓 AI Quiz Generator</h1>
        <span class="role-badge {badge_class}">{role_display}</span>
        <p style="font-size: 1.3rem; margin: 1rem 0 0 0;">Transform educational content into interactive learning experiences</p>
        <p style="font-size: 0.9rem; margin-top: 0.5rem; font-style: italic;">✨ Powered by Multi-Agent AI System • ⚡ Instant Feedback • 🎯 Personalized Quizzes</p>
    </div>
    """, unsafe_allow_html=True)

    # Get and validate API key
    if 'openai_key' not in st.session_state:
        api_key = setup_colab_tunnel()

        if not api_key:
            st.error("❌ OpenAI API key not found in Colab secrets")
            st.markdown("""
            **Setup Required:**
            1. 🔑 Click key icon in Colab sidebar
            2. Add: `OPENAI_API_KEY` = `sk-your-key`
            3. Enable "Notebook access"
            4. **Restart runtime**
            """)
            return

        if not api_key.startswith('sk-'):
            st.error("❌ Invalid API key format")
            return

        # Test and store API key
        try:
            test_client = openai.OpenAI(api_key=api_key)
            test_client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": "test"}],
                max_tokens=1
            )
            st.session_state.openai_key = api_key
        except Exception as e:
            st.error(f"❌ API key validation failed: {str(e)}")
            return

    # Initialize system
    if 'edu_system' not in st.session_state:
        try:
            with st.spinner("🔧 Initializing AI system..."):
                st.session_state.edu_system = HybridEducationalSystem(st.session_state.openai_key)
            st.success("✅ System ready!")
        except Exception as e:
            st.error(f"❌ System initialization failed: {str(e)}")
            return

    # Show setup instructions for Colab users (only for professors)
    if user_role == "professor":
        setup_colab_tunnel()
    
    # Role-based interface
    if user_role == "professor":
        display_professor_interface(st.session_state.edu_system, auth_manager)
        
        # Show created quizzes for professor review
        if 'available_quizzes' in st.session_state and st.session_state.available_quizzes:
            st.markdown("---")
            st.markdown("### 📊 Your Created Quizzes")
            
            for idx, quiz_info in enumerate(st.session_state.available_quizzes):
                quiz = quiz_info['quiz']
                with st.expander(f"📝 {quiz.topic} ({quiz.difficulty_level.value.title()})", expanded=False):
                    col1, col2 = st.columns([3, 1])
                    
                    with col1:
                        st.markdown(f"""
                        **Quiz Details:**
                        - **Questions**: {quiz.total_questions}
                        - **Difficulty**: {quiz.difficulty_level.value.title()}
                        - **Created**: {quiz.created_at.strftime('%Y-%m-%d %H:%M')}
                        - **Source**: {quiz_info['file_name']}
                        - **Cost**: ${quiz_info['metrics']['total_cost']:.4f}
                        """)
                    
                    with col2:
                        if st.button(f"🔍 Preview Quiz", key=f"preview_{idx}"):
                            st.session_state.current_quiz = quiz
                            st.session_state.quiz_mode = "preview"
                            st.rerun()
                        
                        if st.button(f"🗑️ Delete", key=f"delete_{idx}", type="secondary"):
                            st.session_state.available_quizzes.pop(idx)
                            st.rerun()
            
            # Show preview if professor is reviewing
            if st.session_state.get('quiz_mode') == 'preview' and 'current_quiz' in st.session_state:
                st.markdown("---")
                st.markdown("### 🔍 Quiz Preview")
                if st.button("⬅️ Back to Dashboard"):
                    st.session_state.quiz_mode = "dashboard"
                    st.session_state.pop('current_quiz', None)
                    st.rerun()
                display_quiz(st.session_state.current_quiz)
    
    else:  # Student role
        display_student_interface(auth_manager)

if __name__ == "__main__":
    main()

3. Ngrok Tunneling

In [ ]:
from pyngrok import ngrok
from google.colab import userdata
import threading
import subprocess
import time
import os

print("🎓 AI Quiz Generator - Colab Setup")
print("=" * 50)

# Get ngrok token
try:
    ngrok_token = userdata.get('NGROK_TOKEN')
    if not ngrok_token:
        print("❌ Ngrok token not found in Colab secrets")
        print("Add 'NGROK_TOKEN' to your Colab secrets to enable public access")
        print("Get token from: https://dashboard.ngrok.com/get-started/your-authtoken")
    else:
        print("✅ Ngrok token retrieved!")
        ngrok.set_auth_token(ngrok_token)
except Exception as e:
    print(f"❌ Ngrok setup failed: {e}")

# Function to run Streamlit
def run_streamlit():
    os.system("streamlit run education_system.py --server.port 8501 --server.enableCORS false --server.enableXsrfProtection false")

# Start Streamlit in background
thread = threading.Thread(target=run_streamlit)
thread.daemon = True
thread.start()

# Wait for it to start
print("⏳ Starting Streamlit...")
time.sleep(10)

# Create public URL
try:
    if 'ngrok_token' in locals() and ngrok_token:
        print("🌐 Creating public tunnel...")
        public_url = ngrok.connect(8501)

        print("\n" + "="*60)
        print("🎉 SUCCESS! Your AI Quiz Generator is now live!")
        print("="*60)
        print(f"🔗 Public URL: {public_url}")
        print("="*60)

        print("\n📋 Share this URL with your students!")
        print("⚠️  Keep this cell running to maintain the connection!")
    else:
        print("⚠️  Running without public URL (ngrok token missing)")
        print("🔗 Local access: http://localhost:8501")

except Exception as e:
    print(f"❌ Tunnel creation failed: {e}")
    print("🔗 Try accessing locally: http://localhost:8501")